In [1]:
import pandas as pd
from datetime import date, timedelta
from random import randint
from time import sleep
import os
import numpy as np
import pymysql
from sqlalchemy import create_engine
import joblib
import requests
from bs4 import BeautifulSoup
import pulp

## TODO

[x] Change `today + timedelta(days=60)` to `today` on season start date <br>
[ ] Update hardcoded year in "Win % for SoS Calc" after season start <br>
[x] Add Player Cost from SN <br>
[x] Build optimizer <br>
[ ] Confirm where new 2023/2024 data will come from/ live (CSV, DB etc) <br> 

In [2]:
## CONNECT TO DATABASE
username = "root"
password = "root"
host = "localhost"
port = "3307"
database = "nhl_optimizer"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [3]:
query = "SELECT * FROM player_data"

# df = pd.read_sql(query,engine)

### Player Point Value Data (From Sportsnet)

In [4]:
pv_df = pd.read_csv('nhl_player_values.csv')

## Load and Process Models

In [5]:
# Load the models
loaded_goals_pipeline = joblib.load('goals_model.pkl')
loaded_assists_pipeline = joblib.load('assists_model.pkl')

# Read and process the data
season_totals = pd.read_csv('season_totals_23.csv',index_col=0)
season_totals.drop(columns=["Goals/60","Total Assists/60"], inplace=True)



# # Predictions
goal_predictions = loaded_goals_pipeline.predict(season_totals)
assist_predictions = loaded_assists_pipeline.predict(season_totals)

## Functions

In [6]:
from datetime import date, timedelta

def fetch_game_data(year: int):
    """
    Fetches the dates of NHL games and teams involved for a given year.
    
    Parameters:
    - year: The year for which you want to fetch game data.

    Returns:
    - DataFrame with dates, visitor, and home teams.
    """
    
    url = f"https://www.hockey-reference.com/leagues/NHL_{year}_games.html"
    dfs = pd.read_html(url)
    df = dfs[0]
    
    # Convert the "Date" column to datetime.date format
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Extract the relevant columns
    game_data = df[["Date", "Visitor", "Home"]]

    return game_data

def filter_dates_for_week(dates, start_date):
    """
    Filters the provided dates for those that fall within the week starting at start_date.
    
    Parameters:
    - dates: List of dates.
    - start_date: The starting date of the week. 

    Returns:
    - A list of dates that fall within the desired week.
    """
    # Convert start_date to a date object if it's a string
    if isinstance(start_date, str):
        start_date = date.fromisoformat(start_date)
    
    # Calculate the end date of the week
    end_date = start_date + timedelta(days=6)

    # Filter the dates
    week_dates = [d for d in dates if start_date <= d <= end_date]
    
    return week_dates

def opponents_for_team_for_week(year, start_date):
    """
    Fetches the opponents each team faces within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and lists of opponents as values.
    """
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Prepare opponent information
    visitor_opponents = week_games.set_index("Visitor")["Home"].to_dict()
    home_opponents = week_games.set_index("Home")["Visitor"].to_dict()
    
    opponents = {}
    for team in set(week_games["Visitor"].tolist() + week_games["Home"].tolist()):
        opponents[team] = list(set([visitor_opponents.get(team, "")] + [home_opponents.get(team, "")]))

    return opponents


#-------#

def calculate_weekly_team_multipliers(year, start_date, points_df):
    """
    Calculate the weekly multipliers for teams based on their opponents for a given week.
    """
    
    # Fetch the opponents for each team for the week
    games_week = opponents_for_team_for_week(year, start_date)

    weekly_multipliers = {}
    for team, opponents in games_week.items():
        multipliers = []
        for opponent in opponents:
            multiplier_row = points_df[points_df['Team Name'] == opponent]
            if not multiplier_row.empty:
                multipliers.append(multiplier_row['multiplier'].iloc[0])
            else:
                print(f"Warning: Couldn't find multiplier for opponent {opponent}. Skipping...")

        weekly_multipliers[team] = sum(multipliers) / len(multipliers) if multipliers else 1  # Defaulting to 1 if no games

    return weekly_multipliers

# Now, you can create a new column in your players DataFrame to store the composite multiplier for the week:


def games_count_for_team_for_week(year, start_date):
    """
    Fetches the number of games each team plays within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and the number of games they play that week as values.
    """
    
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Count the number of games for each team
    visitor_counts = week_games["Visitor"].value_counts().to_dict()
    home_counts = week_games["Home"].value_counts().to_dict()
    
    total_counts = {}
    for team in set(list(visitor_counts.keys()) + list(home_counts.keys())):
        total_counts[team] = visitor_counts.get(team, 0) + home_counts.get(team, 0)

    return total_counts

## Team Name Dict

In [7]:
# TEAM NAMES
team_abbreviations = {
 'Anaheim Ducks':'ANA',
 'Arizona Coyotes':'ARI',
 'Boston Bruins':'BOS',
 'Buffalo Sabres':'BUF',
 'Calgary Flames':'CGY',
 'Carolina Hurricanes':'CAR',
 'Chicago Blackhawks':'CHI',
 'Colorado Avalanche':'COL',
 'Columbus Blue Jackets':'CBJ',
 'Dallas Stars':'DAL',
 'Detroit Red Wings':'DET',
 'Edmonton Oilers':'EDM',
 'Florida Panthers':'FLA',
 'Los Angeles Kings':'L.A',
 'Minnesota Wild':'MIN',
 'Montreal Canadiens':'MTL',
 'Nashville Predators':'NSH',
 'New Jersey Devils':'N.J',
 'New York Islanders':'NYI',
 'New York Rangers':'NYR',
 'Ottawa Senators':'OTT',
 'Philadelphia Flyers':'PHI',
 'Pittsburgh Penguins':'PIT',
 'San Jose Sharks':'S.J',
 'Seattle Kraken':'SEA',
 'St. Louis Blues':'STL',
 'Tampa Bay Lightning':'T.B',
 'Toronto Maple Leafs':'TOR',
 'Vancouver Canucks':'VAN',
 'Vegas Golden Knights':'VGK',
 'Washington Capitals':'WSH',
 'Winnipeg Jets':'WPG'
}

## Injuries

In [8]:
# Fetch the webpage content
url = "https://www.cbssports.com/nhl/injuries/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Lists to hold our parsed data
teams = []
injury_dataframes = []

# Iterate over each TableBaseWrapper
for wrapper in soup.find_all('div', class_='TableBaseWrapper'):
    
    # Find team name
    team_name_tag = wrapper.find('span', class_='TeamName')
    if team_name_tag:
        team_name = team_name_tag.get_text(strip=True)
        teams.append(team_name)

        # Find the injury table for the team
        table = wrapper.find('table', class_='TableBase-table')
        if table:
            # Use pandas to read the table
            df = pd.read_html(str(table))[0]
            df['Team'] = team_name  # Add a column for the team name
            injury_dataframes.append(df)

# Concatenate all dataframes
injuries_df = pd.concat(injury_dataframes, ignore_index=True)

#Clean the player name up


def clean_player_name(name):
    # Split the name using spaces to identify potential last names
    parts = name.split()

    # The last word (or the two last words in the case of names like "van Riemsdyk") is likely our surname
    surname = parts[-2] + " " + parts[-1] if len(parts) > 2 and len(parts[-2]) <= 3 else parts[-1]

    # Split by surname and get the second part
    full_name = name.split(surname, 1)[1].strip()

    return full_name if full_name else surname

injuries_df['Player'] = injuries_df['Player'].apply(clean_player_name)

## Variables

In [9]:
## VARIABLES

year = 2024


# Get today's date
today = date.today()

## Updated to match new scoring rules for Office Pools
fantasy_goals = 2
fantasy_assists = 1
goaltending_win = 2
ot_so_loss = 0
shutout = 2

# all_dates = fetch_game_dates(year)
# week_dates = filter_dates_for_week(all_dates, today)

#### Win % for SoS Calc

In [10]:
## GATHER WIN % TO USE FOR SOS CALC

dfs = pd.read_html("https://www.hockey-reference.com/leagues/NHL_2023_standings.html", index_col=0)
east_df = dfs[0].reset_index().rename(columns={'index': 'Team Name'})
west_df = dfs[1].reset_index().rename(columns={'index': 'Team Name'})

east_df = east_df.drop([0, 9], axis=0).reset_index(drop=True)
west_df = west_df.drop([0, 9], axis=0).reset_index(drop=True)
east_df['Team Name'] = east_df['Team Name'].str.replace('*', '', regex=False)
west_df['Team Name'] = west_df['Team Name'].str.replace('*', '', regex=False)


east_df = east_df[['Team Name', 'PTS%']]
west_df = west_df[['Team Name', 'PTS%']]

points_df = pd.concat([east_df, west_df], ignore_index=True)
points_df['PTS%'] = points_df['PTS%'].astype(float)


points_df['Abbreviation'] = points_df['Team Name'].map(team_abbreviations)
points_df

,Team Name,PTS%,Abbreviation
0,Boston Bruins,0.823,BOS
1,Toronto Maple Leafs,0.677,TOR
2,Tampa Bay Lightning,0.598,T.B
3,Florida Panthers,0.561,FLA
4,Buffalo Sabres,0.555,BUF
5,Ottawa Senators,0.524,OTT
6,Detroit Red Wings,0.488,DET
7,Montreal Canadiens,0.415,MTL
8,Carolina Hurricanes,0.689,CAR
9,New Jersey Devils,0.683,N.J


## Build Reference DataFrame

In [11]:
# Create dataframe for reference

player_df = pd.DataFrame({
    'Team':season_totals['Team'].values,
    'Position':season_totals['Position'].values,
    'Player':season_totals['Player'].values,
    'Projected Goals/60':goal_predictions,
    'Projected Assists/60':assist_predictions,
    'Projected Goals (TOI)':(season_totals['TOI/GP']/60) * goal_predictions,
    'Projected Assists (TOI)':(season_totals['TOI/GP']/60) * assist_predictions,
})

player_df = player_df.merge(points_df, left_on='Team', right_on='Abbreviation', how='left')

player_df = player_df.round(2).sort_values(by='Projected Goals (TOI)', ascending=False)
player_df['Team'] = player_df['Team Name'].str.split(',').str[0].str.strip()

player_df = player_df[['Team','Abbreviation','Player', 'Position','Projected Goals (TOI)', 'Projected Assists (TOI)']]

player_df = player_df.merge(pv_df[["Team","Player", "pv"]], 
                     on=['Player'], 
                     how='right')

# 3. Select and rename the desired columns
player_df = player_df.rename(columns={
    'Team_x': 'Old Team',
    'Team_y': 'Team',
})

player_df = player_df[['Team', 'Abbreviation', 'Player', 'Position', 'Projected Goals (TOI)', 'Projected Assists (TOI)']]

# Cleaning the team name (remove asterisks and keep only abbreviation, if needed)
points_df['Team'] = points_df['Team Name']

# Calculating multiplier based on points percentage
points_df['multiplier'] = points_df['PTS%'].apply(lambda x: 0.5 / x)



In [12]:
weekly_multipliers = calculate_weekly_team_multipliers(year, today , points_df)
games_this_week = games_count_for_team_for_week(year, today )


# Map this to your players DataFrame
player_df['weekly_multiplier'] = player_df['Team'].map(weekly_multipliers)
player_df['games_this_week'] = player_df['Team'].map(games_this_week)


In [13]:
player_df['projected_goals'] = player_df['Projected Goals (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']
player_df['projected_assists'] = player_df['Projected Assists (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']

# Factoring in fantasy points value for goals and assists
player_df['proj_fantasy_pts'] = (player_df['projected_goals'] * fantasy_goals) + (player_df['projected_assists'] * fantasy_assists)

# Create simple F or D position for Sportsnet
player_df.loc[player_df['Position'] != 'D', 'Position'] = 'F'


In [14]:
# Now, merge on 'Player'
merged_df = pd.merge(player_df, injuries_df[['Player', 'Injury Status']], 
                     on='Player', 
                     how='left')

# Set a new column to indicate if a player is injured or not
merged_df['Injured'] = ~merged_df['Injury Status'].isnull()

# Set the projected fantasy points to 0 for injured players
merged_df['proj_fantasy_pts'] = np.where(merged_df['Injured'], 0, merged_df['proj_fantasy_pts'])

In [15]:
projections_df = merged_df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

projections_df.to_csv('injury_report.csv',index=False)

## Add Team Goaltending Calcs

In [16]:
def estimate_team_goaltending_points(weekly_multipliers, games_this_week, win_points, ot_loss_points, shutout_bonus, avg_ot_loss_freq=0.1, avg_shutout_freq=0.05):
    """
    Estimate the team goaltending points for the week using weekly multipliers, 
    number of games this week, and the fantasy points system.

    Returns:
    - A dictionary with team abbreviations as keys and a tuple of projected goaltending points and games_this_week as values.
    """

    goaltending_data = {}

    for team, multiplier in weekly_multipliers.items():
        games = games_this_week.get(team, 0)

        # Estimate wins based on multiplier (inverse relation)
        projected_wins = games / multiplier
        projected_ot_losses = games * avg_ot_loss_freq  # You can adjust or use team-specific data
        projected_shutouts = games * avg_shutout_freq   # You can adjust or use team-specific data

        total_points = (projected_wins * win_points) + (projected_ot_losses * ot_loss_points) + (projected_shutouts * shutout_bonus)
        
        goaltending_data[team] = (total_points,games)

    return goaltending_data

# Using the function
goaltending_data = estimate_team_goaltending_points(weekly_multipliers, games_this_week, goaltending_win, ot_so_loss, shutout)

# Convert to DataFrame
df = pd.DataFrame(goaltending_data.values(), index=goaltending_data.keys(), columns=['proj_fantasy_pts', 'games_this_week'])

# Reset index to have the teams as a separate column
df = df.reset_index().rename(columns={"index": "Team"})

# Add the position
df["Position"] = "G" 
df['Injured'] = False
df['Player'] = df['Team']

goalie_df = df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

## Final DataFrame Ready for Optimization

In [17]:
# Add goalie df to player df
final_df = pd.concat([projections_df, goalie_df], ignore_index=True).round(2)

## Add Sportsnet player values
final_df = pd.merge(final_df, pv_df[["Team","Player","Position",'pv']], 
                     on=['Team','Player','Position'], 
                     how='left')
final_df.sort_values(by=["pv","proj_fantasy_pts"], ascending=False, inplace=True)
final_df.head(10)

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
252,Colorado Avalanche,False,Nathan MacKinnon,F,2,5.19,12.600
253,Edmonton Oilers,False,Connor McDavid,F,2,4.13,12.500
254,New York Rangers,False,Artemi Panarin,F,2,3.81,11.643
255,Toronto Maple Leafs,False,Auston Matthews,F,2,2.60,11.640
0,Pittsburgh Penguins,False,Erik Karlsson,D,3,4.38,11.500
256,Boston Bruins,False,David Pastrnak,F,2,3.48,11.250
257,Toronto Maple Leafs,False,John Tavares,F,2,2.21,11.000
1,Los Angeles Kings,False,Drew Doughty,D,2,1.18,11.000
258,Toronto Maple Leafs,False,Mitchell Marner,F,2,2.25,10.903
259,Calgary Flames,False,Jonathan Huberdeau,F,2,1.36,10.500


## Optimize Lineup

In [18]:


## OPTIMIZATION RULES

"""
- 6 Forwards MUST be selected
- 4 Defense MUST be selected
- 2 team Goaltenders MUST be selected
- Players are given a salary from 1-5 points
- A team CANNOT go over the salary cap of 30 points
- At least 3 teams MUST be represented in the lineup
"""

MAX_COST = 52.00
NUM_FORWARDS = 6
NUM_DEFENSEMEN = 4 
NUM_GOALIES = 2

final_df = final_df.dropna(subset=['proj_fantasy_pts', 'pv'])

def select_best_team(df, max_cost, num_forwards, num_defensemen, num_goalies):
    # Create a linear programming problem
    prob = pulp.LpProblem("FantasyHockeyTeam", pulp.LpMaximize)

    # Create decision variables
    # Each player gets a binary decision variable
    player_vars = pulp.LpVariable.dicts("player", df.index, cat="Binary")

    # Objective: Maximize total fantasy points
    prob += pulp.lpSum(df['proj_fantasy_pts'][i] * player_vars[i] for i in df.index), "Total Fantasy Points"

    # Constraint: Stay within the salary cap
    prob += pulp.lpSum(df['pv'][i] * player_vars[i] for i in df.index) <= max_cost, "Total Salary Cost"

    # Positional constraints
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'F'].index) == num_forwards, "Number of Forwards"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'D'].index) == num_defensemen, "Number of Defensemen"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'G'].index) == num_goalies, "Number of Goalies"

    # Solve the problem
    prob.solve(pulp.PULP_CBC_CMD())
#      prob.solve(pulp.GLPK_CMD())


    # Extract the selected players
    selected_players = [i for i in df.index if player_vars[i].varValue == 1]
    best_team = df.loc[selected_players]

    return best_team

# Assuming your dataframe is named final_df, and you have defined constants for max_cost, num_forwards, etc.
best_team = select_best_team(final_df, MAX_COST, NUM_FORWARDS, NUM_DEFENSEMEN, NUM_GOALIES)


# Calculate total projected points and total cost for the chosen players from best_team
total_projected_points = sum(best_team.proj_fantasy_pts)
total_cost = sum(best_team.pv)

# Displaying the results

# Create a dataframe with the values
cost = pd.DataFrame({'total_proj_pts': [total_projected_points], 'total_cost': [total_cost]})


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kellen/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2f052ef3ef154467960c7b862bbe8d6a-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/2f052ef3ef154467960c7b862bbe8d6a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 3233 RHS
At line 3238 BOUNDS
At line 3899 ENDATA
Problem MODEL has 4 rows, 660 columns and 1288 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 56.8229 - 0.00 seconds
Cgl0004I processed model has 4 rows, 660 columns (660 integer (660 of which binary)) and 1288 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.71634
Cbc0038I Solution found of -56.3
Cbc0038I Before mini branch and bound, 658 integers at bound fixed and 0 continuous
Cbc0038I Fu

# Lineup

In [19]:
print(cost)
best_team.sort_values(by='Position')

   total_proj_pts  total_cost
0           56.75       51.76


,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
0,Pittsburgh Penguins,False,Erik Karlsson,D,3,4.38,11.500
56,Vegas Golden Knights,False,Shea Theodore,D,3,3.42,5.200
94,Colorado Avalanche,False,Bowen Byram,D,2,2.92,3.850
150,Carolina Hurricanes,False,Tony DeAngelo,D,3,2.95,1.675
270,Vegas Golden Knights,False,Mark Stone,F,3,5.00,9.500
286,Carolina Hurricanes,False,Sebastian Aho,F,3,4.83,8.460
384,Vegas Golden Knights,False,Jonathan Marchessault,F,3,4.79,5.000
450,Carolina Hurricanes,False,Martin Necas,F,3,3.98,3.000
461,Vegas Golden Knights,False,Chandler Stephenson,F,3,3.71,2.750
615,Vegas Golden Knights,False,Pavel Dorofeyev,F,3,4.08,0.825


## Captains

In [20]:
# Find the top players by position based on 'proj_fantasy_pts'
top_players = best_team.loc[best_team.groupby('Position')['proj_fantasy_pts'].idxmax()]

top_players

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
0,Pittsburgh Penguins,False,Erik Karlsson,D,3,4.38,11.5
270,Vegas Golden Knights,False,Mark Stone,F,3,5.00,9.5
740,Nashville Predators,False,Nashville Predators,G,3,8.71,0.0


In [21]:
data = [{'Team': key, 'Multiplier': value} for key, value in weekly_multipliers.items()]
multipliers = pd.DataFrame(data)
multipliers.sort_values(by="Multiplier").round(2)

,Team,Multiplier
20,Nashville Predators,0.71
11,Los Angeles Kings,0.73
2,Arizona Coyotes,0.73
29,Anaheim Ducks,0.73
24,San Jose Sharks,0.75
12,Vancouver Canucks,0.75
26,Columbus Blue Jackets,0.77
30,Ottawa Senators,0.78
21,Detroit Red Wings,0.78
4,St. Louis Blues,0.79


In [22]:
games_this_week

{'Tampa Bay Lightning': 3,
 'Chicago Blackhawks': 3,
 'Arizona Coyotes': 1,
 'New York Rangers': 2,
 'St. Louis Blues': 2,
 'Philadelphia Flyers': 2,
 'Buffalo Sabres': 2,
 'Winnipeg Jets': 2,
 'Calgary Flames': 2,
 'Minnesota Wild': 2,
 'Toronto Maple Leafs': 2,
 'Los Angeles Kings': 2,
 'Vancouver Canucks': 2,
 'Dallas Stars': 1,
 'New Jersey Devils': 2,
 'Seattle Kraken': 3,
 'Montreal Canadiens': 2,
 'Vegas Golden Knights': 3,
 'Colorado Avalanche': 2,
 'Pittsburgh Penguins': 3,
 'Nashville Predators': 3,
 'Detroit Red Wings': 2,
 'New York Islanders': 1,
 'Carolina Hurricanes': 3,
 'San Jose Sharks': 2,
 'Florida Panthers': 2,
 'Columbus Blue Jackets': 2,
 'Edmonton Oilers': 2,
 'Boston Bruins': 2,
 'Anaheim Ducks': 2,
 'Ottawa Senators': 3,
 'Washington Capitals': 1}

In [23]:
injuries_df

,Player,Position,Updated,Injury,Injury Status,Team
0,Alex Killorn,LW,"Sun, Oct 8",Finger,IR. Expected to be out until at least Nov 10,Anaheim
1,Leo Carlsson,C,"Sat, Oct 7",Lower Body,Day-to-Day,Anaheim
2,Chase De Leo,C,"Fri, Oct 6",Knee,IR. Expected to be out until at least Nov 28,Anaheim
3,Isac Lundestrom,C,"Wed, Aug 2",Achilles,Expected to be out until at least Feb 9,Anaheim
4,Shea Weber,D,"Sat, Sep 16",Ankle,IR. Expected to be out until at least Sep 15,Arizona
...,...,...,...,...,...,...
63,Joel Edmundson,D,"Wed, Sep 27",Hand,IR. Expected to be out until at least Oct 29,Washington
64,Max Pacioretty,LW,"Tue, Sep 26",Achilles,IR. Expected to be out until at least Nov 22,Washington
65,Nikolaj Ehlers,LW,"Wed, Oct 4",Neck,Expected to be out until at least Oct 11,Winnipeg
66,Nate Schmidt,D,"Wed, Oct 4",Lower Body,Expected to be out until at least Oct 11,Winnipeg


In [24]:
projections_df

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts
0,Pittsburgh Penguins,False,Erik Karlsson,D,3,4.375244
1,Los Angeles Kings,False,Drew Doughty,D,2,1.175617
2,Columbus Blue Jackets,False,Zach Werenski,D,2,1.487730
3,Boston Bruins,False,Charlie McAvoy,D,2,1.336898
4,Chicago Blackhawks,False,Seth Jones,D,3,3.072289
...,...,...,...,...,...,...
715,Toronto Maple Leafs,False,Toronto Maple Leafs,F,2,NaN
716,New Jersey Devils,False,New Jersey Devils,F,2,NaN
717,Buffalo Sabres,False,Buffalo Sabres,F,2,NaN
718,Los Angeles Kings,False,Los Angeles Kings,F,2,NaN
